In [126]:
import tensorflow as tf 
from tensorflow.keras.optimizers import Adam
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from natsort import natsorted
import decimal

In [98]:
train_data = pd.read_csv("titanic/train.csv")
#train_data = train_data.fillna(0)
#train_data.head()
test_data = pd.read_csv("titanic/test.csv")

y  = train_data["Survived"]
#test_data = test_data.fillna(0)
#train_data['sex'].replace({'male':0, 'female':1}, inplace=True)
# data['sex'].replace({'male':0, 'female':1}, inplace=True)
#print(train_data)
#print(test_data.head())
#women = train_data.loc[train_data.Sex == 'female']["Survived"]
#rate_women = sum(women)/len(women)

In [99]:
def gleanCabin(df):
    df['Sex'].replace({'male':0, 'female':1}, inplace=True)
    col = df["Cabin"]
    desk = col.str.slice(0,1)
    desk = desk.fillna("N")
    room = col.str.slice(1,5).str.extract("([0-9]+)", expand=False).astype("float")
    room = room.fillna(room.mean())
    df["desk"] = desk
    df["room"] = room
gleanCabin(train_data)
gleanCabin(test_data)
drop_elements = ['PassengerId', 'Name', 'SibSp','Parch']
train = train_data.drop(drop_elements, axis = 1)
test = test_data.drop(drop_elements, axis = 1)
#print(train)
#2.find null data and fill new data 
def checkNull_fillData(df):
    for col in df.columns:
        if len(df.loc[df[col].isnull() == True]) != 0:
            if df[col].dtype == "float64" or df[col].dtype == "int64":
                df.loc[df[col].isnull() == True,col] = df[col].mean()
            else:
                df.loc[df[col].isnull() == True,col] = df[col].mode()[0]
                
checkNull_fillData(train)
checkNull_fillData(test)
def prepareTik(df):
    tik = df["Ticket"].str.split()
    destination = []
    fly = []
    for r in  tik:
        if(len(r)>1):
            destination.append(r[0])
            fly.append(r[1])
        else:
            destination.append("N")
            fly.append(r[0])
    df["destination"] = np.array(destination)
    df["fly"] = np.array(fly)
    df.drop("Ticket",axis=1)
prepareTik(train)
prepareTik(test)
#destination = len(tik[0] #.str.get(1)  #.str.split().str.get()
#print(destination,fly)
print(train)
#3.one hot encoding 
str_list = [] 
num_list = []
for colname, colvalue in train.iteritems():
    if type(colvalue[1]) == str:
        str_list.append(colname)
    else:
        num_list.append(colname)
print(str_list) 
print(num_list)
X = pd.get_dummies(train, columns=str_list)

X_test = pd.get_dummies(test, columns=str_list)


     Survived  Pclass  Sex        Age            Ticket     Fare    Cabin  \
0           0       3    0  22.000000         A/5 21171   7.2500  B96 B98   
1           1       1    1  38.000000          PC 17599  71.2833      C85   
2           1       3    1  26.000000  STON/O2. 3101282   7.9250  B96 B98   
3           1       1    1  35.000000            113803  53.1000     C123   
4           0       3    0  35.000000            373450   8.0500  B96 B98   
..        ...     ...  ...        ...               ...      ...      ...   
886         0       2    0  27.000000            211536  13.0000  B96 B98   
887         1       1    1  19.000000            112053  30.0000      B42   
888         0       3    1  29.699118        W./C. 6607  23.4500  B96 B98   
889         1       1    0  26.000000            111369  30.0000     C148   
890         0       3    0  32.000000            370376   7.7500  B96 B98   

    Embarked desk    room destination      fly  
0          S    N   50.49 

In [97]:
X = X.drop(["Survived"],axis=1)
print(X.shape)
print(X_test.shape)

(891, 1556)
(418, 850)


In [87]:
x_train,x_test,y_train,y_test = train_test_split(X,y,
                                                 test_size=0.2,random_state=42)

In [118]:
def modeler(numOflayer,units,dropout,L1,L2):
    inputs = tf.keras.Input(shape=inputShape)
    x = tf.keras.layers.Flatten()(inputs)
    for i in range(numOflayer):
        x = tf.keras.layers.Dense(units, activation = "relu",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=L1, l2=L2))(x)
        x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Dense(1,activation="sigmoid")(x)
    model = Model(inputs,x)
    #model.summary()
    return model


In [141]:
#####   create model   ############

inputShape = 1556
models = [model1,model2,model3,model4,model5,model6,model7,model8,model9,model10]
def create_models(num_models,layers,units,dropout,L1,L2,change,change_range):
    i = 0
    for model in models:
        print(change_range[i])
        if(change=="layers"):
            model=modeler(change_range[i],units,dropout,L1,L2)
        if(change=="units"):
            model=modeler(layers,change_range[i],dropout,L1,L2)
        if(change=="dropout"):
            model=modeler(layers,units,change_range[i],L1,L2)
        if(change=="L1"):
            model=modeler(layers,units,dropout,change_range[i],L2)
        if(change=="L2"):
            model=modeler(layers,units,dropout,L1,change_range[i])
        i= i+1
create_models(models,11,15,0.28,4e-3,1e-3,"L2",np.arange (0e-3, 1e-1, 1e-2))



0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09


In [142]:
eph = 300
lern_rate = 0.0001
opt = Adam(learning_rate=lern_rate, beta_1=0.9, beta_2=0.999, decay=lern_rate/(eph))


#model.compile(optimizer=opt, loss='mse')


In [143]:
i = 0
for model in models:
    i = i + 1
    model.compile(
    optimizer=opt,
    loss=tf.losses.binary_crossentropy,
    metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=eph, verbose=0)
    print("trainning model", i)
    score = model.evaluate(x_train, y_train, verbose=0)
    print(f'Train loss: {score[0]} / Train accuracy: {score[1]}')
    score = model.evaluate(x_test, y_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

trainning model 1
Train loss: 0.38283753395080566 / Train accuracy: 0.932584285736084
Test loss: 0.6798372864723206 / Test accuracy: 0.8491619825363159
trainning model 2
Train loss: 0.18360349535942078 / Train accuracy: 0.9620786309242249
Test loss: 0.8332049250602722 / Test accuracy: 0.8156424760818481
trainning model 3
Train loss: 0.1722472906112671 / Train accuracy: 0.9648876190185547
Test loss: 0.7179688215255737 / Test accuracy: 0.826815664768219
trainning model 4
Train loss: 0.2464035600423813 / Train accuracy: 0.9480336904525757
Test loss: 0.6351896524429321 / Test accuracy: 0.8100558519363403
trainning model 5
Train loss: 0.24427373707294464 / Train accuracy: 0.9452247023582458
Test loss: 0.6313179731369019 / Test accuracy: 0.8435754179954529
trainning model 6
Train loss: 0.23381496965885162 / Train accuracy: 0.9339887499809265
Test loss: 0.9866735339164734 / Test accuracy: 0.826815664768219
trainning model 7
Train loss: 0.19359247386455536 / Train accuracy: 0.9620786309242249


In [137]:
predictions = model.predict(X_test)


ValueError: in user code:

    /home/ubuntu/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py:1586 predict_function  *
        return step_function(self, iterator)
    /home/ubuntu/snap/jupyter/common/lib/python3.7/site-packages/keras/engine/training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ubuntu/snap/jupyter/common/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ubuntu/snap/jupyter/common/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ubuntu/snap/jupyter/common/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ubuntu/snap/jupyter/common/lib/python3.7

In [59]:
predictions = (np.round(predictions.reshape(1,-1)).astype(int))

In [60]:
predictions = (predictions.reshape(418,))

In [61]:
print(predictions)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 0]


In [67]:
output = pd.DataFrame({'PassengerId': test1.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
